In [83]:
import numpy as np
import cv2
from cv2 import aruco
import pandas as pd
import matplotlib.colors as mcolors
# import random

In [84]:
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

In [85]:
def imshow(img, window_name="img"):
    cv2.startWindowThread()
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [86]:
# 鮮鋭化フィルタ
def applyFiltersSp(img, k=2):
    sharp_kernel = np.array([
        [-k / 9, -k / 9, -k / 9],
        [-k / 9, 1 + 8 * k / 9, k / 9],
        [-k / 9, -k / 9, -k / 9]
    ], np.float32)
    img_sp = cv2.filter2D(img, -1, sharp_kernel).astype("uint8")
    return img_sp

# バイラテラルフィルタ
def applyFiltersBltrl(img, d=15):
    img_bltrl = cv2.bilateralFilter(src=img, d=d, sigmaColor=75, sigmaSpace=75)
    return img_bltrl

In [87]:
# 軌跡データ
# csv_path = "output/test/markers_t_hd_60.csv"  # フィルタ適用前
csv_path = "output/test/markers_t_h_60_comlement_filter_bltrl_hybrid_2.csv" # フィルタ適用後
markers_raw = pd.read_csv(csv_path, header=[0, 1])
frames = len(markers_raw)

# 解析で使用するidのデータだけ抜き出し
ids = list(map(lambda col: str(col), [0,1,2,3,4,5,6,7]))
markers = markers_raw[ids]

# 構造（線の接続関係）の定義
stractures = [[0,1,2,3,0],[4,5,6,7,4]]

# 検出したい距離の定義
distancees = [[0,1],[1,2],[2,3],[3,4],[4,0]]

# ストラクチャーに定義されているマーカーidのSeries
stractures_ids = pd.Series(sum(stractures, [])).drop_duplicates().sort_values()


# ids = pd.Series(markers.columns.levels[0])  # カラムのid列を取得
# ids_mask = pd.Series(map(lambda id: str(id).isnumeric(), ids))  # 数値を真、文字列を偽とするマスクを作成
# ids = ids.loc[ids_mask] # マスクを使って数値（id）のみを抜き出し

In [88]:

# 動画データ
cap = cv2.VideoCapture("src/mov/VibrationTest/t_hd_60.mp4")
fps = int(cap.get(cv2.CAP_PROP_FPS))    # 動画のfps
ret, frame = cap.read()
h, w = frame.shape[:2]

# 保存設定
fmt = cv2.VideoWriter_fourcc('m', 'p', '4', 'v') 
writer = cv2.VideoWriter('./output/tracking_test.mp4', fmt, fps, (w, h))

cpxy = ['cpx', 'cpy']
pos = [None, None]
pos_van = [None, None]
pos_lnr = [None, None]
pos_spr = [None, None]
orbits_van = pd.DataFrame()  # 元データの軌跡

font = cv2.FONT_HERSHEY_SIMPLEX 

for id in ids:
    # print("id :", id)
    for i, cp in enumerate(cpxy):
        pos[i] = markers[str(id), cp]
        pos_van[i] = pos[i]
    orbits_van[id] = list(zip(pos_van[0], pos_van[1]))
orbits_van

,0,1,2,3,4,5,6,7
0,"(93, 43)","(460, 59)","(470, 385)","(97, 394)","(97, 440)","(471, 433)","(480, 777)","(100, 786)"
1,"(93, 45)","(461, 54)","(471, 385)","(98, 392)","(97, 439)","(472, 432)","(481, 776)","(100, 783)"
2,"(92, 48)","(462, 51)","(470, 385)","(98, 393)","(97, 437)","(471, 432)","(481, 777)","(99, 781)"
3,"(92, 50)","(461, 48)","(469, 385)","(97, 392)","(97, 436)","(471, 432)","(481, 776)","(100, 782)"
4,"(92, 52)","(461, 48)","(469, 383)","(97, 390)","(97, 435)","(470, 430)","(480, 774)","(99, 779)"
...,...,...,...,...,...,...,...,...
1807,"(88, 58)","(457, 50)","(465, 384)","(94, 392)","(93, 447)","(468, 431)","(480, 774)","(99, 781)"
1808,"(90, 59)","(457, 51)","(466, 385)","(95, 392)","(93, 448)","(468, 431)","(480, 770)","(99, 782)"
1809,"(88, 60)","(458, 51)","(466, 384)","(93, 396)","(93, 449)","(469, 431)","(480, 770)","(99, 783)"
1810,"(88, 60)","(461, 50)","(469, 384)","(93, 400)","(92, 450)","(470, 431)","(481, 775)","(98, 782)"


In [89]:
speed_x = pd.Series([], dtype=float)
speed_y = pd.Series([], dtype=float)
acceleration_x = pd.Series([], dtype=float)
acceleration_y = pd.Series([], dtype=float)

for f in range(frames):
    ret, img = cap.read()
    if not ret or cv2.waitKey(int(1000 / fps)) == 27:
        break

    cv2.putText(img, f"frame: {f}", (10,20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    # img = np.zeros((h, w, 3))

    # フィルター
    img_f = applyFiltersBltrl(img)
    img_f = applyFiltersSp(img_f)

    # マーカー認識
    corners, ids, rejectedImgPoints = aruco.detectMarkers(img_f, dictionary)
    # マーカー可視化
    aruco.drawDetectedMarkers(img, corners, ids, (0, 255, 255))

    # 定義した構造から線を描画（元データ利用）
    positions = np.array(list(orbits_van.iloc[f]))
    lines_from_stractures = []
    for line in stractures:
        for pos in line:
            lines_from_stractures.append(list(orbits_van.iloc[f][str(pos)]))
        # print(lines_from_stractures)
        cv2.polylines(img, np.array([lines_from_stractures]), False, (0,0,255), 2)
        lines_from_stractures = []

    # 座標間の距離を求めて描画
    for ids in distancees:
        pos1_x, pos1_y, pos1_z = [
            markers.iloc[f][(str(ids[0]), 'x')], 
            markers.iloc[f][(str(ids[0]), 'y')], 
            markers.iloc[f][(str(ids[0]), 'z')]
            ]
        pos2_x, pos2_y, pos2_z = [
            markers.iloc[f][(str(ids[1]), 'x')], 
            markers.iloc[f][(str(ids[1]), 'y')], 
            markers.iloc[f][(str(ids[1]), 'z')]
            ]
        distance = np.round(np.sqrt((pos1_x - pos2_x)**2 + (pos1_y - pos2_y)**2 + (pos1_z - pos2_z)**2), 2)
        write_pos = (int((markers.iloc[f][(str(ids[0]), 'cpx')] + markers.iloc[f][(str(ids[1]), 'cpx')])/2),
                     int((markers.iloc[f][(str(ids[0]), 'cpy')] + markers.iloc[f][(str(ids[1]), 'cpy')])/2))
        cv2.putText(img, f"[{ids[0]}]-[{ids[1]}]{distance}m", write_pos, font, 0.5, (0, 255, 255), 1, cv2.LINE_AA)

    # 座標から速度と加速度を求める
    if f > 1:
        for i, id in enumerate(stractures_ids):
            speed_x_current = round(((markers[(str(id), 'x')][f] - markers[(str(id), 'x')][f - 1])) / (fps / 1000), 2)
            if f < 3 : speed_x[i] = 0
            acceleration_x[i] = round((speed_x_current - speed_x[i]) / (fps / 1000), 2)
            speed_x[i] = speed_x_current
            txt = f"[{id}] spd x: {speed_x_current}m/sec | acc x: {acceleration_x[i]}m/sec^2"
            cv2.putText(img, txt, (10,40 + i * 20), font, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    

    # # 通常データを描画（<NA>が入っていれば描画しない）
    # lines_pos_van = np.array(list(orbits_van.iloc[f]))
    # count_nan = pd.Series(list(map(lambda t: pd.Series(t).isna().sum() ,lines_pos_van))).sum()
    # # if count_nan == 0:
    # #     cv2.polylines(img, [lines_pos_van], True, (0,255,255), 2)

    cv2.imshow("mov", img)
    writer.write(img)
    # print(f, lines_pos_spr)

cv2.destroyAllWindows()
writer.release()

In [90]:
positions[0]

array([91, 56])

In [91]:
orbits_van.iloc[f]['4']

(97, 438)